In [52]:
from bs4 import BeautifulSoup
import pandas as pd
import numpy as np
import requests
import json
from json import JSONEncoder

In [120]:
baseUrl = "https://www.vlr.gg"

class HeadToHeadData():
    def __init__(self):
        self.Date = ""
        self.Team1Score = 0
        self.Team2Score = 0

class PastMatchesData():
    def __init__(self):
        self.Team2 = ""
        self.OurTeamScore = 0
        self.Team2Score = 0
        self.Date = ""
        
class PlayerData():
    def __init__(self):
        self.Name = ""
        self.Team = ""
        self.ACS = 0
        self.Kills = 0
        self.Deaths = 0
        self.Assists = 0
        self.KAST = 0
        self.ADR = 0
        self.HeadshotPercent = 0
        self.FirstKills = 0
        self.FirstDeaths = 0

class MapData():
    def __init__(self):
        self.ListOfPlayerDataTeam1 = []
        self.ListOfPlayerDataTeam2 = []
        self.ListOfRounds = []
        self.MapName = "" #All if it is overall stats
        self.MapNumber = 0 #0 if overall stats
        self.Team1Score = 0
        self.Team1Attack = 0
        self.Team2Score = 0
        self.Team2Attack = 0
        self.Pick = ""
        
class MatchData():
    def __init__(self):
        self.Date = ""
        self.Team1 = ""
        self.Team1Score = 0
        self.Team2 = ""
        self.Team2Score = 0
        
        self.ListOfMapData = []
        
        self.ListOfHeadToHead = []
        self.Team1PastMatches = []
        self.Team2PastMatches = []
        
class EmployeeEncoder(JSONEncoder):
        def default(self, o):
            return o.__dict__

In [121]:
response = requests.get("https://www.vlr.gg/event/matches/1015/valorant-champions-2022/?series_id=all&group=completed")

In [122]:
response.status_code

200

In [123]:
eventSoup = BeautifulSoup(response.text)

In [124]:
eventSoup.title.text

'\n\t\t\tValorant Champions 2022: Schedule and Results | Valorant Event | VLR.gg\t\t'

In [125]:
def MatchExactClassName(tag):
    return tag.has_attr('class') and tag['class'] == ['wf-card']

In [126]:
daysMatches = eventSoup.body.find_all(MatchExactClassName)

In [127]:
matchId = 0
mapId = 1
playerDataId = 1
matchDataList = []
mapDataList = []
playerDataList = []

for dayMatches in daysMatches:
    matchesOfTheDay = dayMatches.find_all('a')
    for match in matchesOfTheDay:
        matchId = matchId + 1
        print("Match: " + str(matchId))
        matchSoup = BeautifulSoup(requests.get(baseUrl + match['href']).text)
        
        fileName = str(matchId) + " " + match['href'].split("/")[-1] + ' '.join(matchSoup.find_all("div", {"class": "match-header-date"})[0].text.split())
        
        matchDataList.append([matchId,
                              ' '.join(matchSoup.find_all("div", {"class": "match-header-date"})[0].text.split()),
                             ' '.join(matchSoup.find_all("div", {"class": "wf-title-med"})[0].text.split()),
                             ' '.join(matchSoup.find_all("div", {"class": "wf-title-med"})[1].text.split()),
                             int(' '.join(matchSoup.find_all("div", {"class": "js-spoiler"})[0].find_all("span")[0].text.split())),
                             int(' '.join(matchSoup.find_all("div", {"class": "js-spoiler"})[0].find_all("span")[-1].text.split()))])
            
        
        ############# Map Data
        mapsSoup = matchSoup.find_all("div", {"class": "vm-stats-game"})
        mapNumber = 1
        for mapSoup in mapsSoup:
            mapData = MapData()
            
            ## Map specific details
            if(mapSoup['data-game-id'] != 'all'):
                
                scoreSoupList = mapSoup.find_all("div", {"class": "rnd-currscore"})
                roundList = []
                for scoreSoup in scoreSoupList:
                    if(len(scoreSoup.text.split()) > 0):
                        roundList.append(scoreSoup.text.split()[0].split("-"))
                
                if(len(mapSoup.find_all("div", {"class": "map"})[0].find_all("span")) == 1):
                    mapData.Pick = "Decider"
                    mapDataList.append([
                        mapId,
                        matchId,
                        mapNumber,
                        matchDataList[-1][2],
                        matchDataList[-1][3],
                        mapSoup.find_all("div", {"class": "map"})[0].span.text.split()[0],
                        int(mapSoup.find_all("div", {"class": "score"})[0].text),
                        int(mapSoup.find_all("div", {"class": "score"})[1].text),
                        int(mapSoup.find_all("span", {"class": "mod-t"})[0].text),
                        int(mapSoup.find_all("span", {"class": "mod-t"})[1].text),
                        "Decider",
                        roundList
                    ])
                else:
                    mapDataList.append([
                        mapId,
                        matchId,
                        mapNumber,
                        matchDataList[-1][2],
                        matchDataList[-1][3],
                        mapSoup.find_all("div", {"class": "map"})[0].span.text.split()[0],
                        int(mapSoup.find_all("div", {"class": "score"})[0].text),
                        int(mapSoup.find_all("div", {"class": "score"})[1].text),
                        int(mapSoup.find_all("span", {"class": "mod-t"})[0].text),
                        int(mapSoup.find_all("span", {"class": "mod-t"})[1].text),
                        "Team " + mapSoup.find_all("div", {"class": "map"})[0].span()[0]['class'][1][-1],
                        roundList
                    ])
                
                mapId = mapId + 1
                mapNumber = mapNumber + 1
            
            ## Player stats
            
            playerDataSoup = mapSoup.find_all("table", {"class": "wf-table-inset mod-overview"}) 
            # Team1    playerDataSoup[0]
            team1PlayersSoup = playerDataSoup[0].tbody.find_all("tr")
            for playerSoup in team1PlayersSoup:
                
                statsList = playerSoup.find_all("td", {"class": "mod-stat"})
                
                playerDataList.append([
                    playerDataId,
                    mapId,
                    playerSoup.find_all("div", {"class": "text-of"})[0].text.split()[0],
                    matchDataList[-1][2],
                    playerSoup.find_all("div", {"class": "text-of"})[0].text.split()[0] + matchDataList[-1][2],
                    int(statsList[1].find_all("span", {"class": "mod-both"})[0].text),
                    int(statsList[2].find_all("span", {"class": "mod-both"})[0].text),
                    int(statsList[3].find_all("span", {"class": "mod-both"})[0].text),
                    int(statsList[4].find_all("span", {"class": "mod-both"})[0].text),
                    int(statsList[6].find_all("span", {"class": "mod-both"})[0].text[:-1]),
                    int(statsList[7].find_all("span", {"class": "mod-both"})[0].text),
                    int(statsList[8].find_all("span", {"class": "mod-both"})[0].text[:-1]),
                    int(statsList[9].find_all("span", {"class": "mod-both"})[0].text),
                    int(statsList[10].find_all("span", {"class": "mod-both"})[0].text)
                ])
                playerDataId = playerDataId + 1
            
            # Team2    playerDataSoup[1]
            team2PlayersSoup = playerDataSoup[1].tbody.find_all("tr")
            for playerSoup in team2PlayersSoup:
                
                statsList = playerSoup.find_all("td", {"class": "mod-stat"})
                
                playerDataList.append([
                    playerDataId,
                    mapId,
                    playerSoup.find_all("div", {"class": "text-of"})[0].text.split()[0],
                    matchDataList[-1][3],
                    playerSoup.find_all("div", {"class": "text-of"})[0].text.split()[0] + matchDataList[-1][3],
                    int(statsList[1].find_all("span", {"class": "mod-both"})[0].text),
                    int(statsList[2].find_all("span", {"class": "mod-both"})[0].text),
                    int(statsList[3].find_all("span", {"class": "mod-both"})[0].text),
                    int(statsList[4].find_all("span", {"class": "mod-both"})[0].text),
                    int(statsList[6].find_all("span", {"class": "mod-both"})[0].text[:-1]),
                    int(statsList[7].find_all("span", {"class": "mod-both"})[0].text),
                    int(statsList[8].find_all("span", {"class": "mod-both"})[0].text[:-1]),
                    int(statsList[9].find_all("span", {"class": "mod-both"})[0].text),
                    int(statsList[10].find_all("span", {"class": "mod-both"})[0].text)
                ])
                playerDataId = playerDataId + 1
            
            
matchdf = pd.DataFrame(matchDataList, columns=['id', 'date', 'team1', 'team2', 'team1score', 'team2score'])
mapdf = pd.DataFrame(mapDataList, columns=['id', 'matchId', 'mapNumber', 'team1', 'team2', 'mapName', 'team1score', 'team2score', 'team1attack', 'team2attack', 'pick', 'roundList'])
playerdf = pd.DataFrame(playerDataList, columns=['id', 'mapId', 'name', 'team', 'playerId', 'acs', 'kills', 'deaths', 'assists', 'kast', 'adr', 'headshotPercent', 'fistKills', 'firstDeaths'])

Match: 1
Match: 2
Match: 3
Match: 4
Match: 5
Match: 6
Match: 7
Match: 8
Match: 9
Match: 10
Match: 11
Match: 12
Match: 13
Match: 14
Match: 15
Match: 16
Match: 17
Match: 18
Match: 19
Match: 20
Match: 21
Match: 22
Match: 23
Match: 24
Match: 25
Match: 26
Match: 27
Match: 28
Match: 29
Match: 30
Match: 31
Match: 32
Match: 33
Match: 34


In [128]:
mapdf[:10]

,id,matchId,mapNumber,team1,team2,mapName,team1score,team2score,team1attack,team2attack,pick,roundList
0,1,1,1,LOUD,OpTic Gaming,Ascent,15,13,5,5,Team 1,"[[1, 0], [2, 0], [2, 1], [2, 2], [2, 3], [2, 4..."
1,2,1,2,LOUD,OpTic Gaming,Bind,6,13,4,10,Team 2,"[[0, 1], [0, 2], [0, 3], [0, 4], [1, 4], [2, 4..."
2,3,1,3,LOUD,OpTic Gaming,Breeze,16,14,6,6,Team 1,"[[0, 1], [0, 2], [0, 3], [0, 4], [0, 5], [1, 5..."
3,4,1,4,LOUD,OpTic Gaming,Haven,13,5,6,5,Team 2,"[[1, 0], [2, 0], [3, 0], [3, 1], [3, 2], [4, 2..."
4,5,2,1,OpTic Gaming,DRX,Bind,13,8,7,3,Team 1,"[[0, 1], [0, 2], [1, 2], [2, 2], [3, 2], [4, 2..."
5,6,2,2,OpTic Gaming,DRX,Breeze,13,5,5,4,Team 2,"[[1, 0], [2, 0], [3, 0], [4, 0], [4, 1], [4, 2..."
6,7,2,3,OpTic Gaming,DRX,Fracture,8,13,3,7,Team 1,"[[0, 1], [1, 1], [2, 1], [2, 2], [3, 2], [4, 2..."
7,8,2,4,OpTic Gaming,DRX,Ascent,6,13,3,9,Team 2,"[[0, 1], [0, 2], [0, 3], [0, 4], [1, 4], [1, 5..."
8,9,2,5,OpTic Gaming,DRX,Haven,13,10,5,4,Decider,"[[1, 0], [2, 0], [3, 0], [3, 1], [3, 2], [3, 3..."
9,10,3,1,FunPlus Phoenix,DRX,Icebox,11,13,6,7,Team 1,"[[1, 0], [2, 0], [2, 1], [2, 2], [2, 3], [2, 4..."


In [129]:
matchdf.to_csv("matchData.csv", index=False)
mapdf.to_csv("mapData.csv", index=False)
playerdf.to_csv("playerData.csv", index=False)